<a href="https://colab.research.google.com/github/muchad/23/blob/main/Gen_1_Fail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers==4.4.2
!pip install sentencepiece==0.1.95
!pip install datasets==1.5.0
!pip install nlp==0.4.0
!python -m nltk.downloader punkt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 32.8 MB/s 
     |████████████████████████████████| 880 kB 14.3 MB/s 
     |████████████████████████████████| 3.3 MB 52.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=09c8b1d216ff8849d10c044c20527e1c26ca76ad97b25f5dbbf2f0a63ba7c6bc
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 31.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192 kB 30.4 MB/s 
     |████████████████████████████████| 212 kB 62.4 MB/s 
     |████████████████████████████████| 140 kB 72.9 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My Drive/Colab Notebooks/QG_id_mt5

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/QG_id_mt5


In [2]:
!git clone https://github.com/ozcangundes/multitask-question-generation.git

Cloning into 'multitask-question-generation'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 50 (delta 21), reused 25 (delta 8), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [4]:
%cd multitask-question-generation

/content/gdrive/My Drive/Colab Notebooks/QG_id_mt5/multitask-question-generation


Prepare Data

In [5]:
!python3 prepare_data.py

Downloading: 100% 4.31M/4.31M [00:01<00:00, 2.80MB/s]
Downloading: 100% 99.0/99.0 [00:00<00:00, 87.1kB/s]
Downloading: 100% 82.0/82.0 [00:00<00:00, 70.0kB/s]
100% 323920/323920 [00:08<00:00, 37475.94it/s]
06/04/2022 13:42:15 - INFO - nlp.arrow_writer -   Done writing 323920 examples in 295430510 bytes .
100% 323920/323920 [00:14<00:00, 22578.69it/s]
06/04/2022 13:42:30 - INFO - nlp.arrow_writer -   Done writing 323920 examples in 292472456 bytes .
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100% 324/324 [08:00<00:00,  1.48s/it]
06/04/2022 13:50:30 - INFO - nlp.arrow_writer -   Done writing 323920 examples in 3157224824 bytes .
100% 28348/28348 [00:00<00:00, 36709.67it/s]
06/04/2022 13:50:31 - IN

Fine Tune

In [ ]:
from run_multi import run_multi

args_dict = {
    "model_name_or_path": "google/mt5-small",
    "model_type": "mt5",
    "tokenizer_name_or_path": "mt5_qg_tokenizer",
    "output_dir": "multi",
    "train_file_path": "data/train_data_multitask_mt5.pt",
    "valid_file_path": "data/valid_data_multitask_mt5.pt",
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "gradient_accumulation_steps": 2,
    "learning_rate": 1e-4,
    "num_train_epochs": 2,
    "seed": 42,
    "do_train": True,
    "do_eval": True,
    "evaluate_during_training": True,
    "logging_steps": 100
}

# start training
run_multi(args_dict)

05/30/2022 00:20:02 - WARNING - run_multi -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
05/30/2022 00:20:02 - INFO - run_multi -   Training/evaluation parameters TrainingArguments(output_dir=multi, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=16, gradient_accumulation_steps=2, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/May30_00-20-02_8b0b7f8ffec1, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, logging_steps=100, save_strategy=IntervalStrategy.STEPS, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level=O

In [ ]:
from pipelines import pipeline
pipe = pipeline("question-generation", model = "t5-small-qg-hl")

In [ ]:
text4 = "Forrest Gump is a 1994 American comedy-drama film directed by Robert Zemeckis and written by Eric Roth. \
It is based on the 1986 novel of the same name by Winston Groom and stars Tom Hanks, Robin Wright, Gary Sinise, \
Mykelti Williamson and Sally Field. The story depicts several decades in the life of Forrest Gump (Hanks), \
a slow-witted but kind-hearted man from Alabama who witnesses and unwittingly influences several defining \
historical events in the 20th century United States. The film differs substantially from the novel."

In [ ]:
pipe(text4)

[]